In [33]:
import json
from time import sleep
import requests
import pandas as pd

In [56]:
# 문제정보 한 개만 가져올 때 이용
def get_one_problemData(problem_id):
    """
     solved.ac에서 문제 정보를 받아옵니다.
     too_many_request 에러가 발생하면 중단합니다.
     존재하지 않는 문제의 경우 예외가 발생합니다.
     딕셔너리 형태로 결과를 반환합니다.
    """
    try:
        url = f"https://solved.ac/api/v3/problem/show?problemId={problem_id}"
        r_profile = requests.get(url)
        if r_profile.status_code == requests.codes.too_many_requests: 
            return False
        
        profile = json.loads(r_profile.content.decode('utf-8'))
        tag = ""
        tags = profile.get("tags")
        for t in tags:
            tag += t['key'] +','
            
        problem = \
                {
                    "problemId": profile.get("problemId"),
                    "titleKo": profile.get("titleKo"),
                    "isSolvable": profile.get("isSolvable"),
                    "isPartial" : profile.get("isPartial"),
                    "acceptedUserCount" : profile.get("acceptedUserCount"),
                    "level" : profile.get("level"),
                    "votedUserCount" : profile.get("votedUserCount"),
                    "sprout" : profile.get("sprout"),
                    "givesNoRating" : profile.get("givesNoRating"),
                    "isLevelLocked" : profile.get("isLevelLocked"),
                    "averageTries" : profile.get("averageTries"),
                    "official" : profile.get("official"),
                    "tag" : tag
                }
            
        return problem
    
    except Exception as e:    
        print(f'{problem_id}에서 예외가 발생했습니다.', e)

# id리스트를 받아서 특정 id들의 문제를 받아옴
# 엑셀파일로 저장
def get_problemData_by_list(id_list): 
    """
     solved.ac에서 문제정보를 받아옵니다.
     입력 파라미터 start번부터 end번까지의 문제 정보를 받아옵니다.
     too_many_request 에러가 발생하면 500초 동안 중단합니다.
     df를 반환하고 result 폴더를 생성 후 csv파일로 저장합니다.
    """
    allProblem = []

    for id in id_list:

        result = get_one_problemData(id)
        if result == False: 
            print(f"{id}번째 처리 중 Error 429 발생")
            sleep(500)
            result = get_one_problemData(id)
            allProblem.append(result)
        else:
            allProblem.append(result)

    df = pd .DataFrame()
    for pro in allProblem:
        df = df._append(pro,ignore_index = True)

    df.to_csv("result/data.csv")
    return df
    
# 다수의 문제정보를 받아올 때 이용
def get_many_problemData(start,end):
    """
     start부터 end까지의 범위의 문제의 정보를 가져옵니다.
     tags는 tag의 id만 받습니다. (문제의 장르)
     too many request 에러가 발생하면 500초 동안 일시 중단합니다.
     df를 반환하고 result 폴더를 생성 후 csv파일로 저장합니다.
    """
    try:
        df = pd .DataFrame()
        allData = []
        
        for idx in range(start, end+1,100):
            id = ''
            if end - idx >= 100:
                for num in range(idx,idx+99):
                    id += str(num) + ","
                id += str(idx+99)
            else:
                for num in range(idx,end):
                    id += str(num) + ","
                id += str(end)
            
            url = f"https://solved.ac/api/v3/problem/lookup?problemIds={id}"
            r_profile = requests.get(url)
            if r_profile.status_code == requests.codes.too_many_requests: 
                print(f"{idx}번째 처리 중 Error 429 발생")
                sleep(500)
                url = f"https://solved.ac/api/v3/problem/lookup?problemIds={id}"
                r_profile = requests.get(url)
    
            profile = json.loads(r_profile.content.decode('utf-8'))
            allData = allData + profile
        
        for data in allData:  
            tag = ""
            tags = data["tags"]
            for t in tags:
                tag += t['key'] +','
            data["tags"] = tag
            df = df._append(data,ignore_index = True)
        
        df.to_csv("result/data.csv")
        return df
        
    except Exception as e:    
        print(f'{problem_id}에서 예외가 발생했습니다.', e)

In [52]:
id_list = [1001,1100,1101]
get_problemData_by_list(id_list)

,problemId,titleKo,isSolvable,isPartial,acceptedUserCount,level,votedUserCount,sprout,givesNoRating,isLevelLocked,averageTries,official,tag
0,1001,A-B,True,False,211935,1,16,True,False,True,1.4210,True,"implementation,arithmetic,math,"
1,1100,하얀 칸,True,False,14620,4,72,False,False,False,1.3999,True,"implementation,string,"
2,1101,카드 정리 1,True,False,119,12,12,False,False,False,2.9160,True,"bruteforcing,greedy,"


In [57]:
get_many_problemData(1000,25000)

,problemId,titleKo,titles,isSolvable,isPartial,acceptedUserCount,level,votedUserCount,sprout,givesNoRating,isLevelLocked,averageTries,official,tags,metadata
0,1000,A+B,"[{'language': 'en', 'languageDisplayName': 'en...",True,False,248747,1,37,True,False,True,2.4906,True,"implementation,arithmetic,math,",{}
1,1001,A-B,"[{'language': 'ko', 'languageDisplayName': 'ko...",True,False,211935,1,16,True,False,True,1.4210,True,"implementation,arithmetic,math,",{}
2,1002,터렛,"[{'language': 'ko', 'languageDisplayName': 'ko...",True,False,33220,8,231,False,False,False,4.4786,True,"case_work,geometry,math,",{}
3,1003,피보나치 함수,"[{'language': 'ko', 'languageDisplayName': 'ko...",True,False,45403,8,182,False,False,False,3.0812,True,"dp,",{}
4,1004,어린 왕자,"[{'language': 'ko', 'languageDisplayName': 'ko...",True,False,13610,8,125,False,False,False,2.1913,True,"geometry,math,",{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23268,24996,Album of Numbers,"[{'language': 'en', 'languageDisplayName': 'en...",True,False,1,0,0,False,False,False,1.0000,True,,{}
23269,24997,Well Off,"[{'language': 'en', 'languageDisplayName': 'en...",True,False,3,0,0,False,False,False,1.3333,True,,{}
23270,24998,Accurate Shots (8Mb TL!),"[{'language': 'en', 'languageDisplayName': 'en...",True,False,1,0,0,False,False,False,1.0000,True,,{}
23271,24999,Prom,"[{'language': 'en', 'languageDisplayName': 'en...",True,False,15,8,3,False,False,False,1.3333,True,"binary_search,sorting,",{}
